## Running classifiers from pre set features and problems

In [6]:
from matplotlib import pyplot as plt
from analize_performance import results_all
from run_single_subject import run_logreg_subjects
from src.events_to_annot import valid_patids
import pandas as pd
from bids import BIDSLayout
from sklearn.metrics import ConfusionMatrixDisplay, pair_confusion_matrix

In [26]:
chbmit = '/Volumes/Extreme SSD/EEG_Databases/BIDS_CHB-MIT'
siena = '/Volumes/Extreme SSD/EEG_Databases/BIDS_Siena'
layout = BIDSLayout(chbmit,validate=False)
print(valid_patids[chbmit])


['01', '03', '04', '05', '06', '08', '09', '10', '12', '13', '14', '15', '16', '18', '20', '23', '24']


In [27]:
results = pd.DataFrame(run_logreg_subjects("10",siena))

[PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_Siena/derivatives/linear_features/sub-10/eeg/sub-10_ses-01_task-szMonitoring_run-00_eeg_linear-features.npz'), PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_Siena/derivatives/linear_features/sub-10/eeg/sub-10_ses-01_task-szMonitoring_run-01_eeg_linear-features.npz'), PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_Siena/derivatives/linear_features/sub-10/eeg/sub-10_ses-01_task-szMonitoring_run-02_eeg_linear-features.npz'), PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_Siena/derivatives/linear_features/sub-10/eeg/sub-10_ses-01_task-szMonitoring_run-03_eeg_linear-features.npz'), PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_Siena/derivatives/linear_features/sub-10/eeg/sub-10_ses-01_task-szMonitoring_run-04_eeg_linear-features.npz'), PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_Siena/derivatives/linear_features/sub-10/eeg/sub-10_ses-01_task-szMonitoring_run-05_eeg_linear-features.npz')]
[PosixPath('/Volumes/Extrem

ValueError: Input X contains NaN.
SelectKBest does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [28]:
import numpy as np

In [54]:
features_pat10 = np.load("/Volumes/Extreme SSD/EEG_Databases/BIDS_Siena/derivatives/linear_features/sub-10/eeg/sub-10_ses-01_task-szMonitoring_run-03_eeg_linear-features.npz")

In [55]:
np.isnan(features_pat10["X"]).any()

True

In [57]:
np.isnan(features_pat10["X"]).mean() * 100

0.0008705528830151034

In [62]:
sub14_X = np.load("/Volumes/Extreme SSD/EEG_Databases/BIDS_Siena/derivatives/linear_features/sub-14/eeg/sub-14_ses-01_task-szMonitoring_run-03_eeg_linear-features.npz")

In [63]:
np.isnan(sub14_X["X"]).any()

True

In [65]:
np.isnan(sub14_X["X"]).mean() * 100
np.where(np.isnan(sub14_X["X"]))[0][:10]

array([803, 804])

In [20]:
results = pd.DataFrame([run_logreg_subjects("12",chbmit,mode="prediction",SOP = 25, nseizures_train=2)])

[PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_CHB-MIT/derivatives/linear_features/sub-12/eeg/sub-12_ses-01_task-szMonitoring_run-00_eeg_linear-features.npz'), PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_CHB-MIT/derivatives/linear_features/sub-12/eeg/sub-12_ses-01_task-szMonitoring_run-01_eeg_linear-features.npz'), PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_CHB-MIT/derivatives/linear_features/sub-12/eeg/sub-12_ses-01_task-szMonitoring_run-02_eeg_linear-features.npz'), PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_CHB-MIT/derivatives/linear_features/sub-12/eeg/sub-12_ses-01_task-szMonitoring_run-03_eeg_linear-features.npz'), PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_CHB-MIT/derivatives/linear_features/sub-12/eeg/sub-12_ses-01_task-szMonitoring_run-04_eeg_linear-features.npz'), PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_CHB-MIT/derivatives/linear_features/sub-12/eeg/sub-12_ses-01_task-szMonitoring_run-05_eeg_linear-features.npz'), PosixPath('/Vol

In [21]:
results

,subject,mode,classifier,confusion_matrix,predictions,accuracy,f1,precision,recall,specificity,fpr_hour,PR-AUC,AUC
0,12,prediction,"LogisticRegression(class_weight='balanced', ma...","[[7, 10], [117, 404]]","[0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, ...",0.763941,0.864171,0.975845,0.775432,0.411765,13.3829,0.988776,0.593598


Subject 04 data is problematic and trows an indexing

In [11]:
subject = "12"
event_files = layout.get(
    subject=subject,
     suffix="events",
    extension="tsv",
    return_type="filename"
)
events = [pd.read_csv(f, sep="\t") for f in event_files]
all_events = pd.concat(events, ignore_index=True)
all_events[all_events["eventType"]=="sz"] # the last two seizures are too close together

,onset,duration,eventType,confidence,channels,dateTime,recordingDuration
0,1665.0,61.0,sz,NaN,NaN,2051-01-27 22:44:34,3606.0
1,3415.0,32.0,sz,NaN,NaN,2051-01-27 22:44:34,3606.0
2,1426.0,13.0,sz,NaN,NaN,2051-01-28 00:46:38,3600.0
3,1591.0,23.0,sz,NaN,NaN,2051-01-28 00:46:38,3600.0
4,1957.0,20.0,sz,NaN,NaN,2051-01-28 00:46:38,3600.0
5,2798.0,26.0,sz,NaN,NaN,2051-01-28 00:46:38,3600.0
6,3082.0,32.0,sz,NaN,NaN,2051-01-28 01:46:45,3612.0
7,3503.0,32.0,sz,NaN,NaN,2051-01-28 01:46:45,3612.0
8,593.0,32.0,sz,NaN,NaN,2051-01-28 02:47:45,3611.0
9,811.0,45.0,sz,NaN,NaN,2051-01-28 02:47:45,3611.0


In [24]:
results = pd.DataFrame([run_logreg_subjects( "14",chbmit,mode="detection")])


[PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_CHB-MIT/derivatives/linear_features/sub-14/eeg/sub-14_ses-01_task-szMonitoring_run-00_eeg_linear-features.npz'), PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_CHB-MIT/derivatives/linear_features/sub-14/eeg/sub-14_ses-01_task-szMonitoring_run-01_eeg_linear-features.npz'), PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_CHB-MIT/derivatives/linear_features/sub-14/eeg/sub-14_ses-01_task-szMonitoring_run-02_eeg_linear-features.npz'), PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_CHB-MIT/derivatives/linear_features/sub-14/eeg/sub-14_ses-01_task-szMonitoring_run-03_eeg_linear-features.npz'), PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_CHB-MIT/derivatives/linear_features/sub-14/eeg/sub-14_ses-01_task-szMonitoring_run-04_eeg_linear-features.npz'), PosixPath('/Volumes/Extreme SSD/EEG_Databases/BIDS_CHB-MIT/derivatives/linear_features/sub-14/eeg/sub-14_ses-01_task-szMonitoring_run-05_eeg_linear-features.npz'), PosixPath('/Vol

In [ ]:
all_events = pd.concat(events, ignore_index=True)

valid_patids[chbmit]


In [25]:
results

,subject,mode,classifier,confusion_matrix,predictions,accuracy,f1,precision,recall,specificity,fpr_hour,PR-AUC,AUC
0,14,detection,"LogisticRegression(class_weight='balanced', ma...","[[2846, 1163], [9913, 2359]]","[1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, ...",0.319698,0.298721,0.66979,0.192226,0.709903,51.43173,0.707226,0.451064


In [ ]:
cfm = results.confusion_matrix.values[0]
ConfusionMatrixDisplay(confusion_matrix=cfm).plot()